# Загрузка спутниковых снимков Sentinel-2

Использовалась библиотека sentinelhub

In [193]:
from dotenv import load_dotenv
import os

load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [194]:
from sentinelhub import SHConfig

config = SHConfig()
if CLIENT_ID and CLIENT_SECRET:
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret = CLIENT_SECRET

if config.sh_client_id == '' or config.sh_client_secret == '':
    print("Warning! To use Sentinel Hub services, please provide the credentials (client ID and client secret).")


In [195]:
import numpy as np
import matplotlib.pyplot as plt

from sentinelhub import (
    MimeType,
    CRS,
    BBox,
    SentinelHubRequest,
    SentinelHubDownloadClient,
    DataCollection,
    bbox_to_dimensions,
    DownloadRequest,
    MosaickingOrder,
)

In [196]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [200]:
import datetime

start = datetime.datetime(2021, 6, 1)
end = datetime.datetime(2021, 7, 1)

# Благовещенск
# coords_wgs84 = [ 
#     127.55, 50.15,
#     127.65, 50.25]

# coords_wgs84 = [ 
#     128.35, 51.95,
#     128.45, 52.05]

#coords_wgs84 = [ 127.576115, 50.261766, 127.606946, 50.242737]
# Ярополец
#coords_wgs84 = [35.806246229609144,56.129032544533594,35.84521014799512,56.14813882869631]
# Крымский канал
#coords_wgs84 = [33.63682276130763,46.01458945842896,33.820110969166464,46.153168963731616]

# список координат участков для съемки
start = [127.55, 50.15] #Благовещенск
finish = [128.45, 52.05] #Свободный
current = start
coord_list = []

deltas = [(finish[0] - start[0]) // 0.1 + 1, (finish[1] - start[1])//0.1 + 1]

for y in range(int(deltas[1])):
    for x in range(int(deltas[0])):
        coord_list.append([
            start[0] + 0.1 * x,         start[1] + 0.1 * y, 
            start[0] + 0.1 * (x + 1),   start[1] + 0.1 * (y + 1)])

In [198]:
# RGB бэнды. Реку лучше видно на 8 бэнде инфракрасного но датасет
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""


requests = []
resolution = 10

# адекватные люди бы использовали shapely для задания геометрии реки и загрузки по полученному полигону
# но это не про нас

for coord in coord_list:    
    image_bbox = BBox(bbox= coord, crs=CRS.WGS84)
    image_size = bbox_to_dimensions(image_bbox, resolution=resolution)

    #print(f"Image shape at {resolution} m resolution: {image_size} pixels")

    #для незатопленных регионов
    request_true_color = SentinelHubRequest(
            evalscript=evalscript_true_color,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L1C,
                    time_interval=("2020-05-27", "2020-05-28"),
                    #mosaicking_order=MosaickingOrder.LEAST_CC,
                )
            ],
            responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
            bbox=image_bbox,
            size=image_size,
            config=config,
            data_folder = "unflooded"
        )   
    request_true_color.save_data()

    #для затопленных
    request_true_color = SentinelHubRequest(
            evalscript=evalscript_true_color,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L1C,
                    time_interval=("2021-06-27", "2021-06-28"),
                    #mosaicking_order=MosaickingOrder.LEAST_CC,
                )
            ],
            responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
            bbox=image_bbox,
            size=image_size,
            config=config,
            data_folder = "flooded"
        )   
    request_true_color.save_data()



In [199]:
def plot_image(image, factor=1.0, clip_range=None, **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

    def mini_plot(image, factor=1.0, clip_range=None, **kwargs):
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)